# Data Cleaning and Munging Script - City Data Portal

In [171]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline
pd.options.mode.chained_assignment = None
sns.set()

In [623]:
austin_zips = (78701,
78702,
78703,
78704,
78705,
78717,
78719,
78721,
78722,
78723,
78724,
78725,
78726,
78727,
78728,
78729,
78730,
78731,
78732,
78733,
78734,
78735,
78736,
78737,
78739,
78741,
78742,
78744,
78745,
78746,
78747,
78748,
78749,
78750,
78751,
78752,
78753,
78754,
78756,
78757,
78758,
78759,
78617,
78653,
78660)

### Permits Data

In [624]:
permits_data = pd.read_csv("raw_data/Issued_Construction_Permits.csv")

/Users/carolinekinnen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (54,58,59,60,61,62,63,64,65) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [625]:
permits = permits_data.loc[:,('Permit Num', 'Permit Class Mapped', 
                              'Calendar Year Issued', 'Original Zip')]

permits = permits[permits['Calendar Year Issued'] >= 2011]
permits = permits[permits['Original Zip'].isin(austin_zips)]

res_permits = permits[permits['Permit Class Mapped'] == "Residential"].drop('Permit Class Mapped', axis = 1)
com_permits = permits[permits['Permit Class Mapped'] == "Commercial"].drop('Permit Class Mapped', axis = 1)

In [626]:
res_permits_year_count = res_permits.groupby(['Calendar Year Issued', 'Original Zip']).size().reset_index()
res_permits_year_count.rename(columns = {0: "count", 'Original Zip': 'zip_code', 'Calendar Year Issued': 'year'}, inplace = True)
res_permits_year_count['var_type'] = 'res_permit_count'

In [627]:
com_permits_year_count = com_permits.groupby(['Calendar Year Issued', 'Original Zip']).size().reset_index()
com_permits_year_count.rename(columns = {0: "count", 'Original Zip': 'zip_code', 'Calendar Year Issued': 'year'}, inplace = True)
com_permits_year_count['var_type'] = 'commercial_permit_count'

In [629]:
res_permits_year_count.rename(columns = {'count': 'res_permit_count'}, inplace = True)
res = res_permits_year_count.drop('var_type', axis = 1)

In [631]:
com_permits_year_count.rename(columns = {'count': 'com_permit_count'}, inplace = True)
com = com_permits_year_count.drop('var_type', axis = 1)

In [669]:
all_var_df = pd.merge(com, res, on = ('year', 'zip_code'), how = 'outer')
all_var_df['year'] = all_var_df['year'].astype(str)

### Crime Data

In [637]:
crime_data = pd.read_csv("raw_data/Crime_Reports.csv")

/Users/carolinekinnen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [638]:
crime = crime_data.loc[:,('Incident Number', 'Highest Offense Description', 'Occurred Date Time', 'Zip Code')]

In [639]:
conditions = [
    (crime['Highest Offense Description'].isin(['ASSAULT BY CONTACT', 'ASSAULT WITH INJURY', 'ASSAULT W/INJURY-FAM/DATE VIOL'])),
    (crime['Highest Offense Description'].isin(['AUTO-THEFT', 'BURGLARY NON RESIDENCE', 'BURGLARY OF RESIDENCE', 'BURGLARY OF VEHICLE', 'THEFT', 'THEFT BY SHOPLIFTING'])),
    (crime['Highest Offense Description'].isin(['CRIMINAL MISCHIEF', 'CRIMINAL TRESPASS', 'DISTURBANCE - OTHER', 'FAMILY DISTURBANCE', 'HARASSMENT'])),
    (crime['Highest Offense Description'].isin(['CUSTODY ARREST TRAFFIC WARR', 'WARRANT ARREST NON TRAFFIC'])),
    (crime['Highest Offense Description'].isin(['DWI', 'PUBLIC INTOXICATION'])),
    (crime['Highest Offense Description'].isin(['IDENTITY THEFT'])),
    (crime['Highest Offense Description'].isin(['POSS CONTROLLED SUB/NARCOTIC', 'POSS OF DRUG PARAPHERNALIA', 'POSSESSION OF MARIJUANA'])),
    (crime['Highest Offense Description'].isin(['RUNAWAY CHILD'])),
    ]
     
values = ['assault', 'theft', 'criminal_activity', 'warrent_arrest', 
          'public_intoxication', 'identity_theft', 'narcotics', 'runaway_child']

crime['Offense Category'] = np.select(conditions, values)
crime = crime[crime['Offense Category'] != '0'].drop('Highest Offense Description', axis = 1)

In [640]:
crime.loc[:,'Occurred Date Time'] = pd.to_datetime(crime.loc[:,'Occurred Date Time'])
crime['year'] = crime.loc[:,'Occurred Date Time'].dt.to_period("Y")
crime = crime.drop('Occurred Date Time', axis = 1)

In [641]:
crime_count = crime.groupby(['Offense Category', 'year', 'Zip Code']).size().reset_index()
crime_count['var_type'] = 'count_' + crime_count['Offense Category'] + '_crime'
crime_count.rename(columns = {'Zip Code': 'zip_code', 0:'count'}, inplace = True)
crime_count = crime_count.drop('Offense Category', axis = 1)
crime_count = crime_count[crime_count['year'] >= pd.Period('2011')]
crime_count = crime_count[crime_count['zip_code'].isin(austin_zips)]

In [642]:
crime_pivot = crime_count.pivot(index=["year", "zip_code"], columns="var_type").reset_index()

In [643]:
crime_join = pd.DataFrame(crime_pivot.loc[:, 'year'])
crime_join['zip_code'] = pd.DataFrame(crime_pivot.loc[:,'zip_code'])
crime_join['count_assault_crime'] = crime_pivot.xs('count_assault_crime', level = 'var_type', axis = 1) 

In [644]:
for col in crime_pivot.columns:
    col_name = col[1]
    if col_name != '':
        crime_join[col_name] = crime_pivot.xs(col_name, level = 'var_type', axis = 1)

In [651]:
# 2019 is missing 2 zip codes 
# 2020 is missing 2 zip codes
# 2021 is missing 3 zip codes

#78734 missing from 2019 and 2020 and 2021
#78733 missing from 2021

#zips = crime_join[crime_join['year'] == '2021']['zip_code'].unique()#.groupby('zip_code').count()
#missing = {element for element in austin_zips if element not in zips}
#missing

crime_join = crime_join.append({'year': pd.Period(2019), 'zip_code':78734, 'count_assault_crime':0, 'count_criminal_activity_crime':0,
                  'count_identity_theft_crime': 0, 'count_narcotics_crime': 0, 'count_public_intoxication_crime':0,
                  'count_runaway_child_crime':0, 'count_theft_crime':0, 'count_warrent_arrest_crime':0}, ignore_index = True)


In [721]:
crime_join['zip_code'] = crime_join['zip_code'].astype(str)
crime_join['year'] = crime_join['year'].astype(str)

In [724]:
all_var_df = pd.merge(all_var_df, crime_join, on = ('year', 'zip_code'), how = 'outer')

### Water Consumption Data

In [464]:
water_data_com = pd.read_csv("raw_data/Austin_Water_-_Commercial_Water_Consumption.csv")
water_data_com['var_type'] = 'commercial_water_use_gal'
water_data_res = pd.read_csv("raw_data/Austin_Water_-_Residential_Water_Consumption.csv")
water_data_res['var_type'] = 'res_water_use_gal'

water_data = water_data_res.append(water_data_com)
water_data['year'] = water_data['Year Month'].astype(str).str[0:4]

In [730]:
water = water_data.loc[:,('year', 'Postal Code', 'Total Gallons', 'var_type', 'Customer Class')]
water.rename(columns = {'Postal Code': 'zip_code'}, inplace = True)
water = water[water['zip_code'].isin(austin_zips)]

In [812]:
# reading in from a csv created in R because group by is impossible in Pandas
water_grouped = pd.read_csv("raw_data/water_grouped_sum.csv")

In [813]:
water_grouped.rename(columns = {'postal_code': 'zip_code', 'commercial': 'total_gal_water_com', 'residential': 'total_gal_water_res'}, inplace = True)

In [821]:
water_grouped['zip_code'] = water_grouped['zip_code'].astype(float)
water_grouped['zip_code'] = water_grouped['zip_code'].astype(str)
water_grouped['year'] = water_grouped['year'].astype(str)

In [827]:
all_var_df = pd.merge(all_var_df, water_grouped, on = ('year', 'zip_code'), how = 'outer')

In [828]:
all_var_df

,year,zip_code,com_permit_count,res_permit_count,count_assault_crime,count_criminal_activity_crime,count_identity_theft_crime,count_narcotics_crime,count_public_intoxication_crime,count_runaway_child_crime,count_theft_crime,count_warrent_arrest_crime,total_gal_water_com,total_gal_water_res
0,2011,78617.0,78.0,407.0,66.0,278.0,12.0,32.0,17.0,32.0,184.0,46.0,NaN,NaN
1,2011,78653.0,8.0,4.0,5.0,28.0,4.0,2.0,2.0,1.0,22.0,4.0,NaN,NaN
2,2011,78660.0,70.0,233.0,14.0,29.0,2.0,2.0,19.0,1.0,66.0,20.0,NaN,NaN
3,2011,78701.0,1372.0,55.0,523.0,732.0,40.0,399.0,1583.0,4.0,1283.0,719.0,NaN,NaN
4,2011,78702.0,520.0,1286.0,371.0,1129.0,36.0,752.0,356.0,85.0,1390.0,645.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,2021,78756.0,46.0,200.0,11.0,43.0,14.0,1.0,3.0,NaN,31.0,NaN,NaN,NaN
491,2021,78757.0,100.0,701.0,30.0,96.0,46.0,10.0,17.0,NaN,138.0,NaN,NaN,NaN
492,2021,78758.0,230.0,281.0,147.0,381.0,65.0,28.0,46.0,14.0,304.0,1.0,NaN,NaN
493,2021,78759.0,134.0,517.0,35.0,144.0,62.0,10.0,11.0,4.0,156.0,NaN,NaN,NaN


### 311 Data

In [437]:
three11_data = pd.read_csv("raw_data/311_Unified_Data_Test_Site_2019.csv")

/Users/carolinekinnen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [446]:
conditions = [
    (three11_data['SR Description'].isin(['Animal - Proper Care', 'Animal Control - Assistance Request', 'Found Animal Report - Keep', 'Injured / Sick Animal','Loose Dog','Wildlife Exposure'])),
    (three11_data['SR Description'].isin(['Austin Code - Request Code Officer'])),
    (three11_data['SR Description'].isin(['Graffiti Abatement', 'Public Health - Graffiti Abatement'])),
    (three11_data['SR Description'].isin(['Loud Commercial Music'])),
    (three11_data['SR Description'].isin(['Park Maintenance - Grounds'])),
    (three11_data['SR Description'].isin(['Debris in Street', 'Pothole Repair', 'Sign - Traffic Sign Maintenance', 'Street Light Issue- Address', 'Traffic Signal - Dig Tess Request', 'Traffic Signal - Maintenance'])),
    ]
     
values = ['animal_311', 'request_code_officer_311', 'graffiti_311',
          'loud_commercial_music_311', 'park_maintenance_311', 'road_repair_311']

three11_data['Reporting Category'] = np.select(conditions, values)
three11_small = three11_data[three11_data['Reporting Category'] != '0'].drop('SR Description', axis = 1)

In [470]:
three11_small.loc[:,'Created Date'] = pd.to_datetime(three11_small.loc[:,'Created Date'])
three11_small['year'] = three11_small.loc[:,'Created Date'].dt.to_period("Y")
#crime = crime.drop('Occurred Date Time', axis = 1)

In [840]:
three11 = three11_small[three11_small['year'] >= pd.Period(2011)]
three11 = three11[three11['Zip Code'].isin(austin_zips)]
#three11 = three11.loc[:,('Zip Code', 'Service Request (SR) Number', 'Reporting Category', 'year')]

In [853]:
three11_gb = three11.groupby(['Reporting Category', 'year', 'Zip Code']).size().reset_index()
three11_gb['var_type'] = 'count_' + three11_gb['Reporting Category'] + '_report'
three11_gb.rename(columns = {'Zip Code': 'zip_code', 0:'count'}, inplace = True)
three11_gb = three11_gb.drop('Reporting Category', axis = 1)
three11_gb = three11_gb[three11_gb['zip_code'].isin(austin_zips)]

In [855]:
three11_gb['zip_code'] = three11_gb['zip_code'].astype(str)
three11_gb['year'] = three11_gb['year'].astype(str)

In [866]:
three11_pivot = three11_gb.pivot(index=["year", "zip_code"], columns="var_type").reset_index()
three11_join = pd.DataFrame(three11_pivot.loc[:, 'year'])
three11_join['zip_code'] = pd.DataFrame(three11_pivot.loc[:,'zip_code'])

for col in three11_pivot.columns:
    col_name = col[1]
    if col_name != '':
        three11_join[col_name] = three11_pivot.xs(col_name, level = 'var_type', axis = 1)

In [869]:
all_var_df = pd.merge(all_var_df, three11_join, on = ('year', 'zip_code'), how = 'outer')

In [881]:
all_var_df = all_var_df[all_var_df['year'] != '2021']
all_var_df['zip_code'] = all_var_df['zip_code'].str[0:5]

In [883]:
all_var_df.to_csv("clean_data/city_portal_data.csv")